In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('solution_ex_15').getOrCreate()

22/07/19 12:24:07 WARN Utils: Your hostname, certusr-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/07/19 12:24:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/19 12:24:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/19 12:24:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
inputPath = './data'

inp_movies = inputPath + '/movies.txt'
inp_pref = inputPath + '/preferences.txt'
inp_watched = inputPath + '/watchedmovies.txt'

outputPath = './output_ex_15'

moviesRDD = spark.sparkContext.textFile(inp_movies)
preferencesRDD = spark.sparkContext.textFile(inp_pref)
watchedRDD = spark.sparkContext.textFile(inp_watched)


threshold = 0.5

In [4]:
# Define RDD pairs with keys and userid/gengre as values
moviesRDD = moviesRDD.map(lambda x: (x.split(',')[0], x.split(',')[2]))
watchedRDD = watchedRDD.map(lambda x: (x.split(',')[1], x.split(',')[0]))
preferencesRDD = preferencesRDD.map(lambda x: (x.split(',')[0], x.split(',')[1]))

#moviesRDD.collect()
#watched.collect()

In [5]:
# Join watched movies with movies
moviesRDD_users = watchedRDD.join(moviesRDD)

# Select only userid (as key) and genre (as value) and then
# Cogroup the lists of watched and liked genres for each user
# There is one pair for each userid
# the value contains the list of genres (with repetitions) of the
# watched movies and the list of liked genres
cat_watched_user = moviesRDD_users.map(lambda x: (x[1][0], x[1][1]))
cat_watched_user = cat_watched_user.cogroup(preferencesRDD)

In [6]:
def misleadingProfileFunc(userWatchedLikedGenresLists):
    # Store in a local list the "small" set of liked genres
    # associated with the current user
    likedGenres = list(userWatchedLikedGenresLists[1][1])
    
    # Iterate over the watched movies (the genres of the watched movies)and count
    # - The number of watched movies for this user
    # - How many of watched movies are associated with a not liked genre
    numWatchedMovies = 0
    notLiked = 0
   
    for watchedGenre in userWatchedLikedGenresLists[1][0]:
        numWatchedMovies = numWatchedMovies+1
        if watchedGenre not in likedGenres:
            notLiked = notLiked+1
            
    # Check if the number of watched movies associated with a non-liked genre
    # is greater that threshold%
    if float(notLiked) > threshold * float(numWatchedMovies):
        return True
    else: 
        return False

# Filter the users with a misleading profile
misleadingUsersListsRDD = cat_watched_user.filter(misleadingProfileFunc)

In [7]:
# misleaded_profile_RDD = misleaded_profile_RDD.mapValues(lambda x: (list(x[0]), list(x[1])))

## def getWrongGenre(line):
#     user = line[0]
#     watched_gen = line[1][0]
#     liked_gen = line[1][1]
#     watch = set()
#     for w in watched_gen:
#         if w not in liked_gen:
#             watch.add(w)
# 
#     return (user, list(watch))
# 
# updated_prof = misleaded_profile_RDD.map(lambda x: getWrongGenre(x))

In [8]:
# This function is used in the next transformation to select the pairs (userid,misleading genre)
def misleadingGenresFunc(userWatchedLikedGenresLists):
    # Store in a local list the "small" set of liked genres
    # associated with the current user
    
    userId = userWatchedLikedGenresLists[0]
    likedGenres = list(userWatchedLikedGenresLists[1][1])
    
    # In this solution I suppose that the number of distinct genres for each user 
    # is small and can be stored in a local variable.
    # The local variable is a dictionary that stores for each non-liked genre 
    # also its number of occurrences in the list of watched movies of the current user
    numGenres = {}
    
    
    # Iterate over the watched movies (the genres of the watched movies).
    # Select the watched genres that are not in the liked genres and 
    # count their number of occurrences. Store them in the numGenres dictionary
    for watchedGenre in userWatchedLikedGenresLists[1][0]:
        # Check if the genre is not in the liked ones
        if watchedGenre not in likedGenres:
            # Update the number of times this genre appears
            # in the list of movies watched by the current user
            if watchedGenre in numGenres:
                numGenres[watchedGenre] = numGenres[watchedGenre] + 1
            else:
                numGenres[watchedGenre] = 1
                
    # Select the genres, which are not in the liked ones,
    # which occur at least 5 times
    selectedGenres = []
    
    for genre, occurrences in numGenres.items():
        if occurrences>=5:
            selectedGenres.append( (userId, genre) )
            
    return selectedGenres

# Select the pairs (userid,misleading genre)
misleadingUserGenrePairRDD = misleadingUsersListsRDD.flatMap(misleadingGenresFunc)

In [9]:
misleadingUsersListsRDD.saveAsTextFile(outputPath)